<a href="https://colab.research.google.com/github/BMOnoire/low-resource-machine-translation/blob/master/colab_tamil_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%time
import tensorflow as tf
# some check if tf is allright, if GPU: 0 then check Runtime->Change Runtime->GPU
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.4.0
Num GPUs Available:  1
CPU times: user 282 µs, sys: 0 ns, total: 282 µs
Wall time: 198 µs


In [3]:
%%time
# install stuff and import aachenscript
print("INSTALL fairseq :")
!pip install fairseq
print("\n\n\nINSTALL sacremoses :")
!pip install sacremoses
print("\n\n\nIMPORT trans :")
!git clone https://github.com/GiIbert/trans

INSTALL fairseq :
     |████████████████████████████████| 1.7MB 13.4MB/s 
     |████████████████████████████████| 71kB 9.3MB/s 
     |████████████████████████████████| 133kB 55.7MB/s 
     |████████████████████████████████| 112kB 55.2MB/s 
     |████████████████████████████████| 276kB 47.1MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-cp36-none-any.whl size=141231 sha256=b4e2491dd2c457452dead68821f613ea3ed566a9edfff2b964efdfe707a82240
  Stored in directory: /root/.cache/pip/wheels/e3/e2/fa/b78480b448b8579ddf393bebd3f47ee23aa84c89b6a78285c8
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=06d2928f91ca06802cc6eb5b963780bb189383c6983707c31a9e65fe0f4e4182
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built antlr4-python3-runtime PyYAML
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninst




INSTALL sacremoses :
     |████████████████████████████████| 890kB 13.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=cd8b12eed87d42f78607c4e93e407050a7a44d129a35065dd330a211e295df42
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses



IMPORT trans :
Cloning into 'trans'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 199 (delta 8), reused 82 (delta 7), pack-reused 114
Receiving objects: 100% (199/199), 155.01 MiB | 13.26 MiB/s, done.
Resolving deltas: 100% (52/52), done.
Checking out files: 100% (170/170), done.
CPU times: user 166 ms, sys: 44.9 ms, total: 211 ms
Wall time: 34.6 s


In [4]:
%%time
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
CPU times: user 578 ms, sys: 178 ms, total: 756 ms
Wall time: 25.7 s


In [ ]:
# WARNING: OLD VERSION USE THE SECOND BLOCK
%%time
!CUDA_VISIBLE_DEVICES=0 fairseq-train aachenscript/data-bin/enta \
    --task translation \
    --batch-size 48 \
    --arch transformer --encoder-layers 2 --decoder-layers 2 --share-decoder-input-output-embed  --encoder-attention-heads 2 --decoder-attention-heads 2 --encoder-embed-dim 512 --decoder-embed-dim 512 \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 1e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.25 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096 \
    --tokenizer moses \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --no-epoch-checkpoints \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
    --save-dir drive/MyDrive/tamil_train_checkpoints

/bin/bash: fairseq-train: command not found
CPU times: user 11.6 ms, sys: 3.38 ms, total: 14.9 ms
Wall time: 132 ms


In [ ]:
# YES, THIS ONE
%%time
!CUDA_VISIBLE_DEVICES=0 fairseq-train \
    trans/data-bin/ENTAM \
    --task translation \
    --batch-size 48 \
    --arch transformer --encoder-layers 2 --decoder-layers 2 --share-decoder-input-output-embed --encoder-attention-heads 2 --decoder-attention-heads 2 --encoder-embed-dim 512 --decoder-embed-dim 512 \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 1e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.25 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096 \
    --tokenizer moses \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
    --save-dir drive/MyDrive/tamil_train_checkpoints \
    --keep-last-epochs 1

2021-01-08 16:24:34 | INFO | fairseq_cli.train | Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='transformer', attention_dropout=0.0, batch_size=48, batch_size_valid=48, best_checkpoint_metric='bleu', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='trans/data-bin/ENTAM', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_attention_heads=2, decoder_embed_dim=512, decoder_embed_path=None, decoder_ffn_embed_dim=2048, decoder_input_dim=512, decoder_layerdrop=0, decoder_layers=2, decoder_layers_to_keep=None, decoder_learned_pos=False, decoder_normalize_before=False, decoder_output_dim=512, device_id=0, disable_validation=False, distrib